In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Load the datasets
train_df = pd.read_csv(r"C:\Users\kabil\Downloads\train.csv")
test_df = pd.read_csv(r"C:\Users\kabil\Downloads\test.csv")

# Step 1: Handling Missing Values

# Identify columns with missing values
missing_values_count = train_df.isnull().sum()
print("Missing values in training set:\n", missing_values_count)

# Removing rows with missing target variable (if applicable)
# If 'Survived' is your target variable and cannot be missing
train_df = train_df.dropna(subset=['Survived'])

# Handling missing values using mean imputation for numerical features
# Exclude 'Survived' column from numerical columns
num_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
num_cols.remove('Survived')

# Get numeric columns for the test set as well (excluding target column)
test_num_cols = test_df.select_dtypes(include=[np.number]).columns.tolist()

# Impute missing values in numeric columns
num_imputer = SimpleImputer(strategy='mean')
train_df[num_cols] = num_imputer.fit_transform(train_df[num_cols])
test_df[test_num_cols] = num_imputer.transform(test_df[test_num_cols])

# Handling missing values using the most frequent strategy for categorical features
cat_cols = train_df.select_dtypes(include=['object']).columns.tolist()

# Impute missing values in categorical columns
cat_imputer = SimpleImputer(strategy='most_frequent')
train_df[cat_cols] = cat_imputer.fit_transform(train_df[cat_cols])
test_df[cat_cols] = cat_imputer.transform(test_df[cat_cols])

# Step 2: Handling Outliers

# Define a function to remove outliers based on Z-score
def remove_outliers(df, cols, threshold=3):
    z_scores = np.abs((df[cols] - df[cols].mean()) / df[cols].std())
    return df[(z_scores < threshold).all(axis=1)]

# Apply the function to the numerical columns
train_df = remove_outliers(train_df, num_cols)
test_df = remove_outliers(test_df, test_num_cols)

# Step 3: Encoding Categorical Variables

# Encoding categorical variables using one-hot encoding
train_df = pd.get_dummies(train_df, columns=cat_cols, drop_first=True)
test_df = pd.get_dummies(test_df, columns=cat_cols, drop_first=True)

# Align test set with training set
# Ensure both dataframes have the same columns
test_df = test_df.reindex(columns=train_df.columns, fill_value=0)

# Exclude the target column from the test set
if 'Survived' in test_df.columns:
    test_df = test_df.drop(columns=['Survived'])

# Step 4: Scaling Numerical Features

scaler = StandardScaler()
train_df[num_cols] = scaler.fit_transform(train_df[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols])

# Step 5: Saving the Cleaned Datasets

train_df.to_csv('train_cleaned.csv', index=False)
test_df.to_csv('test_cleaned.csv', index=False)

print("Data cleaning complete. Cleaned datasets saved as 'train_cleaned.csv' and 'test_cleaned.csv'.")


Missing values in training set:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
Data cleaning complete. Cleaned datasets saved as 'train_cleaned.csv' and 'test_cleaned.csv'.
